<a href="https://colab.research.google.com/github/tatooland/tensorflow_v2/blob/master/tensorflowV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow.keras.layers as layers
print(tf.__version__)

##创建简单网络

In [0]:
inputs = tf.keras.Input(shape=(784,), name='img')
h1 = layers.Dense(32, activation='relu')(inputs)
h2 = layers.Dense(32, activation='relu')(h1)
outputs = layers.Dense(10, activation='softmax')(h2)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

model.summary()

tf.keras.utils.plot_model(model, 'mnist_model.png')
tf.keras.utils.plot_model(model, 'model_info.png', show_shapes=True)

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=400, epochs=20, validation_split=.2)
test_scores = model.evaluate(x_test, y_test, verbose=0)

print('test loss:', test_scores[0])
print('test acc:', test_scores[1])

##保存模型和序列化

In [0]:
model.save('model_save.h5')
del model
model = tf.keras.models.load_model('model_save.h5')

#使用共享网络创建多个模型

In [0]:
encode_input = tf.keras.Input(shape=(28, 28, 1), name='img')
h1 = layers.Conv2D(16, 3, activation='relu')(encode_input)
h1 = layers.Conv2D(32, 3, activation='relu')(h1)
h1 = layers.MaxPool2D(3)(h1)
h1 = layers.Conv2D(32, 3, activation='relu')(h1)
h1 = layers.Conv2D(16, 3, activation='relu')(h1)
encode_output = layers.GlobalMaxPool2D(h1)

encode_model = tf.keras.Model(inputs=encode_input, outputs=encode_output)
encode_model.summary()

h2 = layers.Reshape((4, 4, 1))(encode_output)
h2 = layers.Conv2DTranspose(16, 3, activation='relu')(h2)
h2 = layers.Conv2DTranspose(32, 3, activation='relu')(h2)

h2 = layers.UpSampling2D(3)(h2)
h2 = layers.Conv2DTranspose(16, 3, activation='relu')(h2)
decode_output = layers.Conv2DTranspose(1, 3, activation='relu')(h2)

autoencoder = keras.Model(inputs=encode_input, outputs=decode_output)
autoencoder.summary()

#DCGAN

##generator

In [0]:
def generator():
  #target_shape = (H, W, C) = (128, 128, 3)
  
  input_val = tf.keras.Input(shape=(100,), name='noise')
  input = layers.Dense(4 * 4 * 1024, input_shape=(100,), use_bias=False)(input_val)
  input = layers.LeakyReLU()(input)

  g = layers.Reshape((4, 4, 1024))(input)

  g = layers.Conv2DTranspose(512, 3, strides=(2, 2), use_bias=False, padding='same')(g)
  g = layers.BatchNormalization()(g)
  g = layers.LeakyReLU()(g)
  #assert g.shape() == (None, 8, 8, 512)

  g = layers.Conv2DTranspose(256, 3, strides=(2, 2), use_bias=False, padding='same')(g)
  g = layers.BatchNormalization()(g)
  g = layers.LeakyReLU()(g)
  #assert g.shape() == (None, 16, 16, 256)

  g = layers.Conv2DTranspose(128, 3, strides=(2, 2), use_bias=False, padding='same')(g)
  g = layers.BatchNormalization()(g)
  g = layers.LeakyReLU()(g)
  #assert g.shape() == (None, 32, 32, 128)

  g = layers.Conv2DTranspose(64, 3, strides=(2, 2), use_bias=False, padding='same')(g)
  g = layers.BatchNormalization()(g)
  g = layers.LeakyReLU()(g)
  #assert g.shape() == (None, 64, 64, 64)

  output = layers.Conv2DTranspose(3, 3, strides=(2, 2), use_bias=False, activation='tanh', padding='same')(g)
  #assert g.shape() == (None, 128, 128, 3)

  return tf.keras.Model(inputs=input_val, outputs=output)





###生成图片

In [0]:
import PIL
import matplotlib.pyplot as plt
noise = tf.random.normal([1, 100])
g = generator()
g_img = g(noise, training=False)
print(g_img.shape)
plt.imshow(g_img[0, :, :, 0], cmap='gray' )

##DISCRIMINATOR

In [0]:
def discriminator():
  input_val = tf.keras.Input(shape=(128, 128, 3))
  input = layers.Conv2D(64, 3, strides=(2, 2), padding='same')(input_val)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(64, 64, 64)

  input = layers.Conv2D(128, 3, strides=(2, 2), padding='same')(input)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(32, 32, 128)

  input = layers.Conv2D(256, 3, strides=(2, 2), padding='same')(input)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(16, 16, 256)

  input = layers.Conv2D(512, 3, strides=(2, 2), padding='same')(input)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(8, 8, 512)

  input = layers.Conv2D(1024, 3, strides=(2, 2), padding='same')(input)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(4, 4, 1024)

  input = layers.Flatten()(input)
  input = layers.Dense(4*4*1024)(input)
  output = layers.Dense(1)(input)

  return tf.keras.Model(inputs=input_val, outputs=output)

##DISCRIMINATOR LOSS

In [0]:
def combined_loss(d_real_output, d_fake_output):
  bce = tf.keras.losses.BinaryCrossentropy()
  dr_loss = bce(tf.ones(shape=tf.shape(d_real_output), d_real_output)
  df_loss = bce(tf.zeros(shape=d_fake_output), d_fake_output)
  return dr_loss + df_loss

##GENERATOR LOSS